In [1]:
import re
import pandas as pd
from typing import  Literal,List,Any
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.types import Command
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from langchain_core.prompts.chat import ChatPromptTemplate
from langgraph.graph import START, StateGraph,END
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field, field_validator
from langchain_core.messages import HumanMessage,AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv



In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
# print(GROQ_API_KEY)

In [3]:
chat_groq= ChatGroq(model= "llama-3.1-8b-instant")

In [4]:
chat_groq.invoke("hello,what is your name").content

"I don't have a personal name, but I'm an artificial intelligence designed to assist and communicate with users. You can think of me as a conversational AI or a chatbot. I'm here to help answer your questions, provide information, and engage in conversation. How can I assist you today?"

In [5]:
class DateTimeModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4} \d{2}:\d{2}$', v):  # Ensures 'DD-MM-YYYY HH:MM' format
            raise ValueError("The date should be in format 'DD-MM-YYYY HH:MM'")
        return v

In [6]:
class DateModel(BaseModel):
    date: str = Field(description="Properly formatted date", pattern=r'^\d{2}-\d{2}-\d{4}$')

    @field_validator("date")
    def check_format_date(cls, v):
        if not re.match(r'^\d{2}-\d{2}-\d{4}$', v):  # Ensures DD-MM-YYYY format
            raise ValueError("The date must be in the format 'DD-MM-YYYY'")
        return v

In [7]:

class IdentificationNumberModel(BaseModel):
    id: int = Field(description="Identification number (7 or 8 digits long)")
    @field_validator("id")
    def check_format_id(cls, v):
        if not re.match(r'^\d{7,8}$', str(v)):  # Convert to string before matching
            raise ValueError("The ID number should be a 7 or 8-digit number")
        return v

In [8]:
@tool
def check_availability_by_doctor(desired_date:DateModel,doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Checking the database if we have availability for the specific doctor.
    The parameters should be mentioned by the user in the query
    """
    df= pd.read_csv(r'C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv')
    #print(df)
    df['date_slot_time']= df['date_slot'].apply(lambda x:x.split(" ")[-1])
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])
    if len(rows)==0:
        output=  "NO availability of doctor"
    else:
        output= f'this is the availability of doctor in : {desired_date.date} \n'
        output+= "Availbale slots: "+", ".join(rows)
    return output

In [9]:
desired_date = DateModel(date="03-09-2024")

In [10]:
print(check_availability_by_doctor.invoke({
    "desired_date": desired_date,
    "doctor_name": 'john doe'
}))

this is the availability of doctor in : 03-09-2024 
Availbale slots: 08:30, 10:00, 10:30, 11:00, 11:30, 12:00, 13:00, 13:30, 14:00, 14:30, 15:00, 16:30


In [11]:
@tool
def check_availability_by_specilization(desired_date: DateModel, specialization: Literal["general_dentist", "cosmetic_dentist", "prosthodontist", "pediatric_dentist","emergency_dentist","oral_surgeon","orthodontist"]):
    """
    Checking the database if we have availability for the specific specialization.
    The parameters should be mentioned by the user in the query
    """
    df= pd.read_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv")
    df['date_slot_time']= df['date_slot'].apply(lambda x:x.split(' ')[-1])
    rows = df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date) & (df['specialization'] == specialization) & (df['is_available'] == True)].groupby(['specialization', 'doctor_name'])['date_slot_time'].apply(list).reset_index(name='available_slots')
    if len(rows)==0:
        output= "No availability for the desired date and specialization"
    else:
        def convert_to_am_pm(time_str):
            time_str= str(time_str)
            hour,minute= map(int,time_str.split(':'))
            period= 'AM' if hour<12 else 'PM'
            hour= hour%12 or 12
            return f"{hour}:{minute:02d} {period}"
        output = f'This availability for {desired_date.date}\n'
        for row in rows.values:
            output += row[1] + ". Available slots: \n" + ', \n'.join([convert_to_am_pm(value)for value in row[2]])+'\n'

    return output


In [12]:
date_instance = DateModel(date="05-08-2024")
print(date_instance)


date='05-08-2024'


In [13]:
print(check_availability_by_specilization.invoke({
    "desired_date": date_instance, 
    "specialization": "general_dentist"
    }))


This availability for 05-08-2024
john doe. Available slots: 
8:00 AM



In [14]:
@tool
def appointment_cancel(date: DateTimeModel, id: IdentificationNumberModel,doctor_name=Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Cancel an appointment for a given date, ID, and doctor name.
    """
    df= pd.read_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv")
    case_to_remove= df[(df['date_slot']==date.date) & (df['patient_to_attend']==id.id)&(df['doctor_name']==doctor_name)]
    if len(case_to_remove)>0:
        return "You donot have book any appointment with us"
    else:
        df.loc[(df['date_slot']==date.date) & (df['patient_to_attend']==id.id)&(df['doctor_name']==doctor_name),['is_available','patient_to attend']]=[True, None]
        df.to_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\data\doctor_availability.csv",index=False)
        return "Successfully Cancelled the appointment"

In [15]:
Date = DateTimeModel(date="07-08-2024 08:30")
Date

DateTimeModel(date='07-08-2024 08:30')

In [16]:

IDNumber = IdentificationNumberModel(id=1000097)
IDNumber

IdentificationNumberModel(id=1000097)

In [17]:
IdentificationNumberModel(id=1000097)


IdentificationNumberModel(id=1000097)

In [18]:
print(appointment_cancel.invoke({"date": Date,"id":IDNumber,"doctor_name":"john doe"}))


Successfully Cancelled the appointment


In [ ]:
@tool
def reschedule_appointments(old_date: DateTimeModel, new_date: DateTimeModel,id_number: IdentificationNumberModel,doctor_name: Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Reschedule appointments for a given patient.
    """
    df= pd.read_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv")
    available_for_desired_date= df[(df['date_slot']==new_date.date)&(df['is_available']==True)& (df['doctor_name']==doctor_name)]
    if len(available_for_desired_date)==0:
        return "Sorry, the doctor is not available for the desired date."
    else:
        cancel_appointment.invoke({'date':old_date, 'id_number':id_number, 'doctor_name':doctor_name})
        set_appointment.invoke({'desired_date':new_date, 'id_number': id_number, 'doctor_name': doctor_name})
        return "Successfully rescheduled for the desired time"


In [20]:
@tool
def set_appointment(desired_date:DateTimeModel, id_number:IdentificationNumberModel, doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Set appointment or slot with the doctor.
    The parameters MUST be mentioned by the user in the query.
    """
    df= pd.read_csv(r"C:\Users\yogass\Desktop\doctor_appointment_multiagent\notebook\availability.csv")

   
    from datetime import datetime
    def convert_datetime_format(dt_str):
        # Parse the input datetime string
        #dt = datetime.strptime(dt_str, "%Y-%m-%d %H:%M")
        dt = datetime.strptime(dt_str, "%d-%m-%Y %H:%M")
        
        # Format the output as 'DD-MM-YYYY H.M' (removing leading zero from hour only)
        return dt.strftime("%d-%m-%Y %#H.%M")
    
    case = df[(df['date_slot'] == convert_datetime_format(desired_date.date))&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]
    if len(case) == 0:
        return "No available appointments for that particular case"
    else:
        df.loc[(df['date_slot'] == convert_datetime_format(desired_date.date))&(df['doctor_name'] == doctor_name) & (df['is_available'] == True), ['is_available','patient_to_attend']] = [False, id_number.id]
        df.to_csv(f"../data/doctor_availability.csv", index = False)

        return "Succesfully done"

In [21]:

Date = DateTimeModel(date="07-08-2024 08:30")
Date

DateTimeModel(date='07-08-2024 08:30')

In [22]:
DateTimeModel(date='07-08-2024 08:30')


DateTimeModel(date='07-08-2024 08:30')

In [23]:
IDNumber = IdentificationNumberModel(id=1000097)
IDNumber

IdentificationNumberModel(id=1000097)

In [24]:
IdentificationNumberModel(id=1000097)


IdentificationNumberModel(id=1000097)

In [25]:
print(set_appointment.invoke({"desired_date":Date,"id_number":IDNumber,"doctor_name":"john doe"}))


No available appointments for that particular case


In [26]:
class Router(TypedDict):
    next: Literal["information_node","booking_node","FINISH"]
    reasoning: str

In [27]:
class AgentState(TypedDict):
    messages: Annotated[list[Any], add_messages]
    id_number: int
    next:str
    query: str
    current_reasoning: str

In [28]:
members_dict = {'information_node':'specialized agent to provide information related to availability of doctors or any FAQs related to hospital.','booking_node':'specialized agent to only to book, cancel or reschedule appointment'}


In [29]:
members_dict

{'information_node': 'specialized agent to provide information related to availability of doctors or any FAQs related to hospital.',
 'booking_node': 'specialized agent to only to book, cancel or reschedule appointment'}

In [30]:
options = list(members_dict.keys()) + ["FINISH"]


In [31]:
options

['information_node', 'booking_node', 'FINISH']

In [32]:
worker_info = '\n\n'.join([f'WORKER: {member} \nDESCRIPTION: {description}' for member, description in members_dict.items()]) + '\n\nWORKER: FINISH \nDESCRIPTION: If User Query is answered and route to Finished'


In [34]:
print(worker_info)

WORKER: information_node 
DESCRIPTION: specialized agent to provide information related to availability of doctors or any FAQs related to hospital.

WORKER: booking_node 
DESCRIPTION: specialized agent to only to book, cancel or reschedule appointment

WORKER: FINISH 
DESCRIPTION: If User Query is answered and route to Finished


In [35]:
system_prompt = (
    "You are a supervisor tasked with managing a conversation between following workers. "
    "### SPECIALIZED ASSISTANT:\n"
    f"{worker_info}\n\n"
    "Your primary role is to help the user make an appointment with the doctor and provide updates on FAQs and doctor's availability. "
    "If a customer requests to know the availability of a doctor or to book, reschedule, or cancel an appointment, "
    "delegate the task to the appropriate specialized workers. Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
    "UTILIZE last conversation to assess if the conversation should end you answered the query, then route to FINISH "
     )


In [36]:
print(system_prompt)

You are a supervisor tasked with managing a conversation between following workers. ### SPECIALIZED ASSISTANT:
WORKER: information_node 
DESCRIPTION: specialized agent to provide information related to availability of doctors or any FAQs related to hospital.

WORKER: booking_node 
DESCRIPTION: specialized agent to only to book, cancel or reschedule appointment

WORKER: FINISH 
DESCRIPTION: If User Query is answered and route to Finished

Your primary role is to help the user make an appointment with the doctor and provide updates on FAQs and doctor's availability. If a customer requests to know the availability of a doctor or to book, reschedule, or cancel an appointment, delegate the task to the appropriate specialized workers. Given the following user request, respond with the worker to act next. Each worker will perform a task and respond with their results and status. When finished, respond with FINISH.UTILIZE last conversation to assess if the conversation should end you answered 

In [37]:
def supervisor_node(state:AgentState) -> Command[Literal['information_node', 'booking_node', '__end__']]:
    print("**************************below is my state right after entering****************************")
    print(state)
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"user's identification number is {state['id_number']}"},
    ] + state["messages"]
    
    print("***********************this is my message*****************************************")
    print(messages)
    
    # query = state['messages'][-1].content if state["messages"] else ""
    query = ''
    if len(state['messages']) == 1:
        query = state['messages'][0].content
      
    print("************below is my query********************")    
    print(query)
    
    response = chat_groq.with_structured_output(Router).invoke(messages)
    
    goto = response["next"]
    
    print("********************************this is my goto*************************")
    print(goto)
    
    print("********************************")
    print(response["reasoning"])
          
    if goto == "FINISH":
        goto = END
        
    print("**************************below is my state****************************")
    print(state)
    
    if query:
        return Command(goto=goto, update={'next': goto, 
                                          'query': query, 
                                          'current_reasoning': response["reasoning"],
                                          'messages': [HumanMessage(content=f"user's identification number is {state['id_number']}")]
                        })
    return Command(goto=goto, update={'next': goto, 
                                      'current_reasoning': response["reasoning"]}
                   )
